# 从全连接到卷积

## 分类猫和狗的图片

- 使用一个还不错的相机采集图片（12M像素）
- RTB图片有36M元素
- 使用100大小的单隐藏层MLP，模型有3.6B元素
  - 远多于世界上所有猫和狗的总数（900M狗，600M猫）

## 回顾：单隐藏层MLP
3.6B参数=14GB
## 例子：Waldo在哪里

在图片里面找Waldo

- 两个原则
  - 平移不变性
  - 局部性

## 重新考察全连接层
- 将输入和输出变形为矩阵（宽度，高度）
- 将权重变形为4-D张量$(h,w)到(h',w')$ $$h_{i,j}=\sum_{k,l}{w_{i,j,k,l}x_{k,l}}=\sum_{a,b}{v_{i,j,a,b}x_{i+a, j+b}} $$
- V是W的重新索引$v_{i,j,a,b}=w_{i,j,i+a,j+b}$

## 原则 #1-平移不变性
- x的平移导致h的平移$h_{i,j}=\sum_{a,b}{v_{i,j,a,b}x_{i+a, j+b}}$
- v不应依赖于(i,j)
- 解决方案：令$v_{i,j,a,b}=v_{a,b}$ $$h_{i,j}=\sum_{a,b}{v_{a,b}x_{i+a, j+b}}$$
- 这就是2维 ~~卷积~~ 交叉相关

## 原则 #2-局部性
$$h_{i,j}=\sum_{a,b}{v_{a,b}x_{i+a, j+b}}$$
- 当评估$h_{i,j}$时，我们不应该用远离$x_{i,j}$的参数
- 解决方案：当$|a|,|b|>\Delta时，使得v_{a,b}=0$ $$h_{i,j}=\sum_{a=-\Delta}\sum_{b=-\Delta}{v_{a,b}x_{i+a, j+b}} $$

## 总结
- 对全连接层使用平移不变性和局部性得到卷积层
$$
\begin{align}
h_{i,j}&=\sum_{a,b}{v_{i,j,a,b}x_{i+a, j+b}}\\
&=\sum_{a,b}{v_{a,b}x_{i+a, j+b}}\\
&=\sum_{a=-\Delta}\sum_{b=-\Delta}{v_{a,b}x_{i+a, j+b}}\\
\end{align}
$$

# 卷积层

二维交叉相关

$$Input*Kernel=Output$$
## 二维卷积层
- 输入$X: n_h \times n_w$
- 核$W: k_h \times k_w$
- 偏差$b \in \mathbb{R}$
- 输出$Y: (n_h - k_h +1)\times(n_w - k_w +1)$即：$$Y=X*W+b$$
- $W$ 和 $b$是可学习的参数

## 例子

原始图片(矩阵)*核K

$$\begin{align}
K&=\left[\begin{matrix}
-1& -1& -1\\
-1&  8& -1\\
-1& -1& -1
\end{matrix}\right]\rightarrow 边缘检测\\
K&=\left[\begin{matrix}
 0& -1&  0\\
-1&  5& -1\\
 0& -1&  0
\end{matrix}\right]\rightarrow 锐化\\
K&=\frac{1}{16}\left[\begin{matrix}
1& 2& 1\\
2& 4& 2\\
1& 2& 1
\end{matrix}\right]\rightarrow 高斯模糊
\end{align}
$$

## 交叉相关和卷积
- 二维交叉相关$$y_{i,j}=\sum_{a=1}^{h}\sum_{b=1}^{w}{W_{a,b}x_{i+a, j+b}}$$
- 二维卷积$$y_{i,j}=\sum_{a=1}^{h}\sum_{b=1}^{w}{W_{-a,-b}x_{i+a, j+b}}$$
- 由于对称性，在实际使用中没有区别

## 一维和三维交叉相关
- 一维$$y_{i}=\sum_{a=1}^{h}{W_{a}x_{i+a}}$$
  - 文本
  - 语言
  - 时序序列
- 三维$$y_{i,j,k}=\sum_{a=1}^{h}\sum_{b=1}^{w}\sum_{c=1}^{d}{W_{a,b,c}x_{i+a, j+b, k+c}}$$
  - 视频
  - 医学图像
  - 气象地图

## 总结
- 卷积层将输入和核矩阵进行交叉相关，加上偏移之后得到输出
- 核矩阵和偏移是可学习的参数
- 核矩阵的大小是超参数

# 图像卷积

In [1]:
import torch
from torch import nn
import d2l_source.d2l as d2l

二维互相关

In [2]:
def corr2d(X, K): 
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

验证输出

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

实现二维卷积层

In [4]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

## 简单应用，检测不同颜色的边缘

In [5]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

### 预设核矩阵

In [6]:
K = torch.tensor([[1.0, -1.0]])

### 检测输出

1代表白色到黑色的边缘，-1代表黑色到白色的边缘

In [7]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

### 只能检测垂直的边缘

In [8]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

## 学习由`X`生成`Y`的卷积核

In [17]:
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # 学习率

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 4.998
epoch 4, loss 1.568
epoch 6, loss 0.562
epoch 8, loss 0.217
epoch 10, loss 0.087


所学到的卷积核权重张量

In [18]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9626, -1.0228]])